In [ ]:
#Kindly Note : if import Fails PIP install corresponding library
import json
import requests
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords
import numpy
from gensim import corpora

In [ ]:
def FindAlternateGroups_v4(url):
    def GetTitlesLinks(url_link):
            #Set the product collection common URL string
            text1 = 'collections/all/products.json?page='
            #Initialize lists of Titles and Links to products for final JSON array
            all_titles = []
            all_links = []
            #We assume there are MAX 50 pages of products to iterate over each page, this value can be changed
            for i in range(1,50):
                #Strng for Final URL to each product
                final_url = url_link + text1 + str(i)
                print(final_url)
                #Request JSON data
                response = requests.get(final_url)
                #Load JSON into variable
                response_json = json.loads(response.text)
                #Select PRODUCTS from JSON for further analysis
                products_json = response_json['products']
                #If PRODUCTS not empty
                if products_json:
                    #Iterate products to get their title and links
                    for pattern in products_json:
                        title = pattern['title']
                        links = pattern['handle']
                        all_titles.append(title)
                        all_links.append(links)
                else:
                    break
            #print(all_titles)
            #print(all_links)
            return all_titles, all_links
    
    def clean_sentence(sentence, stopwords=False):
        #Perform text pre-processing and cleaning for Bag of Words
        sentence = sentence.lower().strip()  #Convert to lower case
        sentence = re.sub(r'[^a-z0-9\s]', '', sentence) #Remove special chars
        
        if stopwords:
            #The Gensim pre-built function to remove all the stop words if set TRUE
            sentence = remove_stopwords(sentence)
        return sentence
                        
    def get_cleaned_sentences(df,stopwords=False):    
        #Empty an array of cleaned Titles
        cleaned_sentences=[]
        #Iterate to get cleaned titles
        for index in range(len(df)):
            #print(index,row)
            cleaned=clean_sentence(df[index],stopwords)
            cleaned_sentences.append(cleaned)
        return cleaned_sentences
    
    def retrieveSimilarProducts(question, question_embedded,sentence_embeddings,sentences, max_sim):
        #Receives the title, title Embedding vectors, original title string and max_sim to filterout Alternate/Similar products
        index_sim=-1
        #Dict for storing individual product alternates
        dict1 = {}
        var = question
        var_2 = url_link + "/products/"
        #List to store links to Laternate products
        li = []
        #Interate over each title and get similarity index, store if has it exceeds max_sim
        for index,pattern_embedding in enumerate(sentence_embeddings):
            sim=cosine_similarity(pattern_embedding,question_embedded)[0][0];
            #print(index, sim, sentences[index])
            if (sim>max_sim):
                li.append(var_2+links[index])
        dict1.update({var:li})        
        #print(li)
        #print(dict1)
        return dict1
    
    url_link = url
    titles, links = GetTitlesLinks(url_link)
    cleaned_sentences_with_stopwords=get_cleaned_sentences(titles,stopwords=False)
    #print(cleaned_sentences_with_stopwords)
    sentences=cleaned_sentences_with_stopwords
    # The next piece of code splits the sentences by white spaces 
    sentence_words = [[word for word in document.split() ]
            for document in sentences]
    #Create a new Corpus Dictionary for all the words
    dictionary = corpora.Dictionary(sentence_words)
    bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
    #List to hold the final JSON for output
    dict_main=[]
    #Final iteration, clubbing everything
    for i in range(len(titles)):
        a = clean_sentence(titles[i],stopwords=False)
        a = dictionary.doc2bow(a.split())
        c = retrieveSimilarProducts(titles[i],a, bow_corpus, titles, max_sim=0.95)
        dict_main.append(c)
    print (dict_main)
    #output to a JSON file
    with open("output_v2.json", "w") as outfile:
        json.dump(dict_main, outfile)

In [ ]:
"""Execute this cell last. Change the string of text to the desired URL and code will do the need for you.
"""
text = "https://www.woolsboutiqueuomo.com/"
FindAlternateGroups_v4(url=text)